## (1) Data extraction

Extract text contained in json files and save it in a dataframe for further pre-processing.

In [72]:
%%capture
from tqdm.notebook import tqdm
tqdm.pandas()
import json
import argparse
import pandas as pd
import cudf
import numpy as np

ImportError: No module named 'cudf'

In [3]:
# Check the different keys in the json file
for id_file in tqdm(range(1,14)):
    file_path = "./../Data/Original_data/" + str(id_file) + ".json"

    with open(file_path) as f:
        data = json.load(f)

        # Loop over each document
        keys = []
        for i, doc in enumerate(data):
            for key, value in doc.items():
                keys.append(key)

        myset = set(keys)
        print("File {} - Keys: {} - Documents: {}".format(id_file, myset, len(data)))

File 1 - Keys: {'text', 'uri'} - Documents: 11699
File 2 - Keys: {'text', 'uri'} - Documents: 32072
File 3 - Keys: {'text', 'uri'} - Documents: 8225
File 4 - Keys: {'text', 'uri'} - Documents: 77258
File 5 - Keys: {'text', 'uri'} - Documents: 46079
File 6 - Keys: {'text', 'uri'} - Documents: 28106
File 7 - Keys: {'text', 'uri'} - Documents: 27391
File 8 - Keys: {'text', 'uri'} - Documents: 24143
File 9 - Keys: {'text', 'uri'} - Documents: 22223
File 10 - Keys: {'text', 'uri'} - Documents: 20979
File 11 - Keys: {'text', 'uri'} - Documents: 57160
File 12 - Keys: {'text', 'uri'} - Documents: 85900
File 13 - Keys: {'text', 'uri'} - Documents: 793



In [4]:
rows = []
for id_file in tqdm(range(1, 14)):
    file_path = "./../Data/Original_data/" + str(id_file) + ".json"

    with open(file_path) as f:
        print("Loading file {}...".format(id_file))
        data = json.load(f)  # data is a list of dict of the form: {'text':['...'], 'uri':['...']}

        print("Extracting text from {} documents in file...".format(len(data), id_file))
        for i, doc in enumerate(data):
            text = doc.get('text') # Get the text of the current doc
            if text is not None:
                row_dict = {'Text': text[0]}
                rows.append(row_dict)

# Create dataframe
df = pd.DataFrame(rows)
df

Loading file 1...
Extracting text from 11699 documents in file...
Loading file 2...
Extracting text from 32072 documents in file...
Loading file 3...
Extracting text from 8225 documents in file...
Loading file 4...
Extracting text from 77258 documents in file...
Loading file 5...
Extracting text from 46079 documents in file...
Loading file 6...
Extracting text from 28106 documents in file...
Loading file 7...
Extracting text from 27391 documents in file...
Loading file 8...
Extracting text from 24143 documents in file...
Loading file 9...
Extracting text from 22223 documents in file...
Loading file 10...
Extracting text from 20979 documents in file...
Loading file 11...
Extracting text from 57160 documents in file...
Loading file 12...
Extracting text from 85900 documents in file...
Loading file 13...
Extracting text from 793 documents in file...



,Text
0,"Americas Headquarters: Cisco Systems, Inc., 17..."
1,For the latest version of the Cisco Small Busi...
2,"WebEx Meeting Center User Guide For Hosts, Pr..."
3,78-4019959-01 Rev D Prisma II 1550 nm SuperQA...
4,Release Notes for Cisco RV130/RV130W Firmware ...
...,...
442014,Cisco Unified Communications System for IP Tel...
442015,Cisco Unified Communications System for IP Tel...
442016,Cisco Unified Communications System for IP Tel...
442017,Cisco Unified Communications System for IP Tel...


## (2) Data Processing

In order to use "create_pretraining_data.py" from BERT repository, the input must be a plain text file, with one sentence per line and one blank line between documents:

  * One sentence per line. These should ideally be actual sentences, not entire paragraphs or arbitrary spans of text. (Because we use the sentence boundaries for the "next sentence prediction" task).
  * Blank lines between documents. Document boundaries are needed so that the "next sentence prediction" task doesn't span between documents.
  
They advise to perform sentence segmentation with an off-the-shelf NLP toolkit such as spaCy.

In [ ]:
# Sample df for testing
sample_df = df.copy(deep=True)
sample_df = sample_df.head()

### 2.1. Corpus Cleaning

In [56]:
sample_df.Text = sample_df.Text.replace('\s+', ' ', regex=True)  # Remove duplicate spaces
sample_df.Text = sample_df.Text.str.encode('ascii', 'ignore').str.decode('utf-8')   # Encode in ascii to remove weird characters such as \uf0a7

### 2.2 Sentence Segmentation

In [69]:
# Sentence segmentation with spacy
import spacy
nlp = spacy.load('en_core_web_sm')

def sent_segmentation(doc_text):
    """
    """
    doc = nlp(doc_text)
    sentences = list(doc.sents)
    sentences = [sent.text for sent in sentences]
    sentences = "\n".join(sentences)
    return sentences

sentences = sent_segmentation(sample)
print(sentences)

### 2.3 Sentence Cleaning

In [ ]:
# If line begins with a number, remove the number

# If line has more than 10 punctuations, remove line

# If line has only 2 words, remove it